In [29]:
import numpy as np

In [30]:
import tensorflow as tf
import os
import cv2
import random

In [31]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
# import matplotlib.pyplot as plt

In [32]:
seed_constant = 15
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [33]:
image_height, image_width = 64, 64
max_images_per_class = 8000
 
dataset_directory = "dataset"

classes_list = ["Fighting","Shooting","RoadAccidents","Robbery","Abuse","Arrest","Arson","Assault","Burglary","Explosion","Normal"]

 
model_output_size = len(classes_list)

In [34]:
model_output_size

11

In [35]:
def frames_extraction(video_path):
    # Empty List declared to store video frames
    frames_list = []
     
    # Reading the Video File Using the VideoCapture
    video_reader = cv2.VideoCapture(video_path)
 
    # Iterating through Video Frames
    while True:
 
        # Reading a frame from the video file 
        success, frame = video_reader.read() 
 
        # If Video frame was not successfully read then break the loop
        if not success:
            break
 
        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
        # print(video_path)
         
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
         
        # Appending the normalized frame into the frames list
        frames_list.append(normalized_frame)
     
    # Closing the VideoCapture object and releasing all resources. 
    video_reader.release()
 
    # returning the frames list 
    return frames_list

In [13]:
# import moviepy.editor as mp

In [14]:
# def resize(video_path):

 
#         # Resize the Frame to fixed Dimensions
#     clip = mp.VideoFileClip(video_path)
#     clip_resized = clip.resize(height=64)
#     clip_resized.write_videofile(video_path)

    
    

In [15]:
# path="C:/Users/rudra/Downloads/cctv_analysis/dataset/Anomaly-Videos-Part-3"
# for classes in classes_list:
#     temp_path=os.path.join(path,classes)
#     for videos in os.listdir(temp_path):
#         print(videos)
#         video_path=os.path.join(temp_path,videos)
#         print(video_path)
#         resize(video_path)

In [9]:
def create_dataset():
 
    # Declaring Empty Lists to store the features and labels values.
    temp_features = [] 
    features = []
    
    labels = []
    dataset_directory = "C:/Users/rudra/Downloads/cctv_2/dataset"
     
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(classes_list):
        print(f'Extracting Data of Class: {class_name}')
         
        # Getting the list of video files present in the specific class name directory
        files_list = os.listdir(os.path.join(dataset_directory, class_name))
 
        # Iterating through all the files present in the files list
        for file_name in files_list:
 
            # Construct the complete video path
            video_file_path = os.path.join(dataset_directory, class_name, file_name)
            
 
            # Calling the frame_extraction method for every video file path
#             resize(video_file_path)
            frames = frames_extraction(video_file_path)
            print(video_file_path)
 
            # Appending the frames to a temporary list.
            temp_features.extend(frames)
         
        # Adding randomly selected frames to the features list
        features.extend(random.sample(temp_features, max_images_per_class))
 
        # Adding Fixed number of labels to the labels list
        labels.extend([class_index] * max_images_per_class)
         
        # Emptying the temp_features list so it can be reused to store all frames of the next class.
        temp_features.clear()
 
    # Converting the features and labels lists to numpy arrays
    # features = np.asarray(features)
    # labels = np.array(labels)  
    # a = np.asarray(features)
    np.save("features.npy",features)
    # b = np.asarray(labels)
    np.save("labels.npy",labels)
 
    # return features, labels

In [10]:
# features, labels = create_dataset()
create_dataset()

Extracting Data of Class: Fighting
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting002_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting003_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting004_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting005_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting006_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting007_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting008_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting009_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting010_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting011_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting012_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting013_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighting014_x264.mp4
C:/Users/rudra/Downloads/cctv_2/dataset\Fighting\Fighti

In [36]:
features = np.load("features.npy")
labels = np.load("labels.npy")

In [ ]:
import pandas as pd

In [37]:
from tensorflow.keras.utils import to_categorical
one_hot_encoded_labels = to_categorical(labels)

In [38]:
print(labels)

[ 0  0  0 ... 10 10 10]


In [39]:
from sklearn.model_selection import train_test_split

In [40]:
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = seed_constant)


In [41]:
features_test.shape

(17600, 64, 64, 3)

In [42]:
def create_model():
 
    # We will use a Sequential model for model construction
    model = Sequential()
 
    # Defining The Model Architecture
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = (image_height, image_width, 3)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(model_output_size, activation = 'softmax'))
 
    # Printing the models summary
    model.summary()
 
    return model
 
 
# Calling the create_model method
model = create_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 batch_normalization (BatchN  (None, 60, 60, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0         
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 64)               0         
 lobalAveragePooling2D)                                          
                                                        

In [43]:
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)
 
# Adding loss, optimizer and metrics values to the model.
model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
 
# Start Training
model_training_history = model.fit(x = features_train, y = labels_train, epochs = 20, batch_size = 4 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])


Epoch 1/20
14080/14080 [==============================] - 682s 48ms/step - loss: 1.6696 - accuracy: 0.4594 - val_loss: 1.5941 - val_accuracy: 0.6149
Epoch 2/20
14080/14080 [==============================] - 655s 47ms/step - loss: 1.0261 - accuracy: 0.6885 - val_loss: 1.3060 - val_accuracy: 0.6880
Epoch 3/20
14080/14080 [==============================] - 667s 47ms/step - loss: 0.7885 - accuracy: 0.7638 - val_loss: 1.5330 - val_accuracy: 0.7391
Epoch 4/20
14080/14080 [==============================] - 666s 47ms/step - loss: 0.6658 - accuracy: 0.8025 - val_loss: 2.4630 - val_accuracy: 0.6765
Epoch 5/20
14080/14080 [==============================] - 664s 47ms/step - loss: 0.5899 - accuracy: 0.8260 - val_loss: 0.8009 - val_accuracy: 0.8541
Epoch 6/20
14080/14080 [==============================] - 655s 46ms/step - loss: 0.5331 - accuracy: 0.8428 - val_loss: 0.9090 - val_accuracy: 0.8317
Epoch 7/20
14080/14080 [==============================] - 630s 45ms/step - loss: 0.4999 - accuracy: 0.8550

In [44]:
print(model_training_history)

In [45]:
model.save('model_8000.h5')

In [46]:
model = tf.keras.models.load_model('model_8000.h5')

In [47]:
model_evaluation_history = model.evaluate(features_test, labels_test)

550/550 [==============================] - 31s 57ms/step - loss: 0.7197 - accuracy: 0.9109


In [48]:
def plot_metric(metric_name_1, metric_name_2, plot_name):
  # Get Metric values using metric names as identifiers
  metric_value_1 = model_training_history.history[metric_name_1]
  metric_value_2 = model_training_history.history[metric_name_2]
 
  # Constructing a range object which will be used as time 
  epochs = range(len(metric_value_1))
   
  # Plotting the Graph
  plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
  plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)
   
  # Adding title to the plot
  plt.title(str(plot_name))
 
  # Adding legend to the plot
  plt.legend()

In [20]:
plot_metric('loss', 'val_loss', 'Total Loss vs Total Validation Loss')

NameError: name 'model_training_history' is not defined

In [49]:
from collections import deque

In [50]:
# def save_video(frames):
    

In [51]:
def save_clip(frames):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  
    frame_rate=30
    output_video = cv2.VideoWriter("C:/Users/rudra/Downloads/output_clip.mp4", fourcc, frame_rate, (width, height))

    for frame in frames:
        output_video.write(frame)
    output_video.release()
 



In [80]:
def predict_on_live_video(video_file_path, output_file_path, window_size):
    frame_count=0
    frames=[]
    # Initialize a Deque Object with a fixed size which will be used to implement moving/rolling average functionality.
    predicted_labels_probabilities_deque = deque(maxlen = window_size)
 
    # Reading the Video File using the VideoCapture Object
#     video_reader = cv2.VideoCapture(video_file_path)
    video_reader = cv2.VideoCapture(video_file_path)
 
    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))
 
    while True: 
 
        # Reading The Frame
        status, frame = video_reader.read() 
 
        if not status:
            break
 
        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
         
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
 
        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis = 0))[0]
 
        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)
 
        # Assuring that the Deque is completely filled before starting the averaging process
        if len(predicted_labels_probabilities_deque) == window_size:
 
            # Converting Predicted Labels Probabilities Deque into Numpy array
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)
 
            # Calculating Average of Predicted Labels Probabilities Column Wise 
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)
 
            # Converting the predicted probabilities into labels by returning the index of the maximum value.
            predicted_max=np.max(predicted_labels_probabilities_averaged)
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)
            print(predicted_max)
            if predicted_max>0.6 and predicted_label<10:
                predicted_label = np.argmax(predicted_labels_probabilities_averaged)
                frame_count=frame_count+1
                if frame_count<450:
                    frames.append(frame)
                else:
                    if len(frames)>100:
                        save_clip(frames)
                    frame_count=0
                    frames=[]
            else:
                predicted_label=10 
            # Accessing The Class Name using predicted label.
            predicted_class_name = classes_list[predicted_label]

           
            # Overlaying Class Name Text Ontop of the Frame
            # cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            print(predicted_class_name)
 
        # Writing The Frame
        video_writer.write(frame)
 
 
        cv2.imshow('Predicted Frames', frame)
 
        key_pressed = cv2.waitKey(10)
 
        if key_pressed == ord('q'):
            break
 
    cv2.destroyAllWindows()
 
     
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()

In [81]:
output_file_path = "C:/Users/rudra/OneDrive/Desktop"
# output_file_path = 0
window_size = 30
video_file_path = "C:/Users/rudra/Downloads/cctv_2/dataset/Explosion/Explosion003_x264.mp4"
predict_on_live_video(video_file_path, output_file_path, window_size)

1/1 [==============================] - 0s 23ms/step
0.9263074
Explosion
1/1 [==============================] - 0s 24ms/step
0.9252349
Explosion
1/1 [==============================] - 0s 32ms/step
0.9242799
Explosion
1/1 [==============================] - 0s 32ms/step
0.9249436
Explosion
1/1 [==============================] - 0s 24ms/step
0.92448676
Explosion
1/1 [==============================] - 0s 24ms/step
0.92398745
Explosion
1/1 [==============================] - 0s 27ms/step
0.92312175
Explosion
1/1 [==============================] - 0s 24ms/step
0.9238346
Explosion
1/1 [==============================] - 0s 24ms/step
0.9239246
Explosion
1/1 [==============================] - 0s 24ms/step
0.9244405
Explosion
1/1 [==============================] - 0s 24ms/step
0.92631346
Explosion
1/1 [==============================] - 0s 27ms/step
0.92779756
Explosion
1/1 [==============================] - 0s 24ms/step
0.92304224
Explosion
1/1 [==============================] - 0s 24ms/step
0.9186